In [ ]:
import sys
sys.path.append("..")
from config import aws_key, aws_secret
from config import mongo_username, mongo_password, mongo_server_name
from config import postgres_server_name, postgres_database, postgres_username, postgres_port, postgres_password

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
db_url = f"postgres://{postgres_username}:{postgres_password}@{postgres_server_name}:{postgres_port}/{postgres_database}"
engine = create_engine(db_url)
conn = engine.connect()
session = Session(bind=engine)

# Crime

In [ ]:
query = session.query(' * from all_crime')
crime_df = pd.read_sql(query.statement, engine)


In [ ]:
crime_df['report_date'] = pd.to_datetime(crime_df['report_date'])
crime_df.head()

In [ ]:
def crime_variable (row):
    if row['crime_type'] == 'HOMICIDE':
        return 1
    if row['crime_type'] == 'ROBBERY-PEDESTRIAN':
        return 1
    if row['crime_type'] == 'BURGLARY-RESIDENCE':
        return 1
    if row['crime_type'] == 'AGG ASSAULT':
        return 1
    return 0

# applying the function to go through every row and create a new column called 'rainy'

crime_df['person_crime'] = crime_df.apply(lambda row:crime_variable(row),axis = 1)

In [8]:
crime_df['year'] = pd.DatetimeIndex(crime_df['report_date']).year
crime_df['month'] = pd.DatetimeIndex(crime_df['report_date']).month

In [9]:
crime_person_df = crime_df[["person_crime","year", "month"]]

In [10]:
crime_person_time_df = crime_person_df.groupby(["year", "month"]).sum().reset_index()
crime_person_time_df.head()

,year,month,person_crime
0,2009,1,1125
1,2009,2,777
2,2009,3,920
3,2009,4,944
4,2009,5,993


# Weather

In [ ]:
query = session.query(' * from hist_weather')
atl_weather_df = pd.read_sql(query.statement, engine)


In [ ]:
atl_weather_df['date_simple'] = pd.to_datetime(atl_weather_df['date_simple'])
atl_weather_df.head()

In [ ]:
atl_weather_df['year'] = pd.DatetimeIndex(atl_weather_df['date_simple']).year
atl_weather_df['month'] = pd.DatetimeIndex(atl_weather_df['date_simple']).month

In [ ]:
real_feel_Time_df = atl_weather_df[["feels_like","year", "month"]]
real_feel_TimeGroup_df = real_feel_Time_df.groupby(["year", "month"]).mean().reset_index()
real_feel_TimeGroup_df.head()

# Join Crime and Weather

In [ ]:
personCrime_temp_df = pd.merge(crime_person_time_df,real_feel_TimeGroup_df, how = 'inner', on = ['year','month'])

In [ ]:
personCrime_temp_df["day"] = 1

In [ ]:
personCrime_temp_df["date"] = pd.to_datetime(personCrime_temp_df[['year', 'month', 'day']])

In [ ]:
org_crime_temp_df = personCrime_temp_df[["date","person_crime","feels_like"]]
org_crime_temp_df = org_crime_temp_df.set_index("date")

In [ ]:
org_crime_temp_df.head()

In [ ]:
# write to mongo
import tra_functions as tralib
tralib.write_df_to_mongo_as_json("Ethan_Crime_Weather_Exploration", org_crime_temp_df)

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Date')
ax1.set_ylabel('Count Crimes', color=color)
ax1.plot(org_crime_temp_df.index, org_crime_temp_df.person_crime, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Real Feel', color=color)  # we already handled the x-label with ax1
ax2.plot(org_crime_temp_df.index, org_crime_temp_df.feels_like, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()